In [2]:
from google.colab import files
files.upload()

Saving clenched_data.csv to clenched_data.csv
Saving fist_data.csv to fist_data.csv
Saving four_data.csv to four_data.csv
Saving index_finger_data.csv to index_finger_data.csv
Saving okay_data.csv to okay_data.csv
Saving peace_data.csv to peace_data.csv
Saving rest_data.csv to rest_data.csv
Saving rock_data.csv to rock_data.csv
Saving spread_data.csv to spread_data.csv
Saving three_data.csv to three_data.csv
Saving thumb_data.csv to thumb_data.csv
Saving up_data.csv to up_data.csv


{'clenched_data.csv': b'ch1,ch2,timestamp_ms,label\n549,525,0,clenched\n534,488,10,clenched\n513,529,20,clenched\n502,514,30,clenched\n526,490,40,clenched\n540,526,50,clenched\n548,501,60,clenched\n504,515,70,clenched\n548,516,80,clenched\n501,521,90,clenched\n500,497,100,clenched\n528,500,110,clenched\n521,481,120,clenched\n529,501,130,clenched\n533,514,140,clenched\n547,517,150,clenched\n549,502,160,clenched\n541,481,170,clenched\n534,493,180,clenched\n534,505,190,clenched\n505,489,200,clenched\n510,503,210,clenched\n519,483,220,clenched\n527,511,230,clenched\n539,495,240,clenched\n546,488,250,clenched\n533,518,260,clenched\n538,507,270,clenched\n511,511,280,clenched\n518,499,290,clenched\n517,513,300,clenched\n527,506,310,clenched\n545,508,320,clenched\n525,505,330,clenched\n540,524,340,clenched\n544,483,350,clenched\n546,526,360,clenched\n508,518,370,clenched\n514,518,380,clenched\n507,526,390,clenched\n531,482,400,clenched\n549,503,410,clenched\n513,515,420,clenched\n530,491,430,c

In [30]:
import pandas as pd
import numpy as np
import os
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [31]:
def rms(signal):
    return np.sqrt(np.mean(np.square(signal)))

def mav(signal):
    return np.mean(np.abs(signal))

def zc(signal):
    return np.sum(np.diff(np.sign(signal)) != 0)

def wl(signal):
    return np.sum(np.abs(np.diff(signal)))

def ssc(signal):
    return np.sum(np.diff(np.sign(np.diff(signal))) != 0)

def extract_features(window):
    features = []
    for ch in ['ch1', 'ch2']:
        x = window[ch].values
        features.extend([
            rms(x), mav(x), zc(x), wl(x), ssc(x),
            np.mean(x), np.std(x), np.var(x)
        ])
    return features


In [32]:

csv_files = sorted(glob("*.csv"))
dataframes = [pd.read_csv(f) for f in csv_files]
data = pd.concat(dataframes, ignore_index=True)


In [33]:
window_size = 100
stride = 50

X = []
y = []

for label in data['label'].unique():
    gesture_df = data[data['label'] == label].reset_index(drop=True)
    for start in range(0, len(gesture_df) - window_size + 1, stride):
        window = gesture_df.iloc[start:start + window_size]
        features = extract_features(window)
        X.append(features)
        y.append(label)


feature_names = [f'{stat}_{ch}' for ch in ['ch1', 'ch2']
                 for stat in ['RMS', 'MAV', 'ZC', 'WL', 'SSC', 'Mean', 'Std', 'Var']]
features_df = pd.DataFrame(X, columns=feature_names)
features_df['label'] = y


features_df.to_csv("time_domain_features.csv", index=False)


print("First 5 rows of extracted feature dataset:")
features_df


First 5 rows of extracted feature dataset:


,RMS_ch1,MAV_ch1,ZC_ch1,WL_ch1,SSC_ch1,Mean_ch1,Std_ch1,Var_ch1,RMS_ch2,MAV_ch2,ZC_ch2,WL_ch2,SSC_ch2,Mean_ch2,Std_ch2,Var_ch2,label
0,526.149028,525.94,0,1651.0,67,525.94,14.829579,219.9164,505.485578,505.29,0,1686.0,71,505.29,14.060082,197.6859,clenched
1,524.173674,523.98,0,1726.0,72,523.98,14.247793,202.9996,505.577986,505.38,0,1760.0,69,505.38,14.147636,200.1556,clenched
2,523.925395,523.72,0,1784.0,73,523.72,14.669070,215.1816,504.137670,503.93,0,1693.0,72,503.93,14.468763,209.3451,clenched
3,524.777886,524.53,0,1860.0,69,524.53,16.127898,260.1091,504.481288,504.25,0,1786.0,73,504.25,15.274407,233.3075,clenched
4,523.940283,523.70,0,1636.0,58,523.70,15.866001,251.7300,507.993878,507.78,0,1766.0,67,507.78,14.739457,217.2516,clenched
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,524.347633,524.16,0,1442.0,65,524.16,14.026204,196.7344,507.632012,507.44,0,1622.0,69,507.44,13.960888,194.9064,up
1448,525.197430,525.00,0,1646.0,69,525.00,14.399306,207.3400,504.427051,504.23,0,1623.0,70,504.23,14.098124,198.7571,up
1449,524.238524,524.03,0,1767.0,73,524.03,14.784759,218.5891,504.954988,504.74,0,1803.0,71,504.74,14.733377,217.0724,up
1450,524.411299,524.21,0,1622.0,72,524.21,14.528796,211.0859,504.271802,504.05,0,1935.0,71,504.05,14.954849,223.6475,up


In [34]:
df = pd.read_csv("time_domain_features.csv")

print("Number of samples per gesture class:\n")
print(df['label'].value_counts())


Number of samples per gesture class:

label
clenched        121
fist            121
four            121
index_finger    121
okay            121
peace           121
rest            121
rock            121
spread          121
three           121
thumb           121
up              121
Name: count, dtype: int64


In [39]:
features_df = features_df.dropna()

X = features_df.drop('label', axis=1).values
y = features_df['label'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)
print(y_train.shape)


(998, 16)
(998,)


In [40]:

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)


KNeighborsClassifier()

In [41]:
y_pred_lr = lr.predict(X_test)
y_pred_svm = svm.predict(X_test)
y_pred_knn = knn.predict(X_test)

acc_lr = accuracy_score(y_test, y_pred_lr) * 100
acc_svm = accuracy_score(y_test, y_pred_svm) * 100
acc_knn = accuracy_score(y_test, y_pred_knn) * 100

print(f"Logistic Regression: {acc_lr:.2f}%")
print(f"KNN (k=5):           {acc_knn:.2f}%")
print(f"SVM (RBF Kernel):    {acc_svm:.2f}%")



Logistic Regression: 9.32%
KNN (k=5):           10.72%
SVM (RBF Kernel):    11.19%
